In [1]:
import pandas as pd
import numpy as np
import matplotlib as mlib
import seaborn as sns
%matplotlib inline
import re

In [2]:
train = pd.read_excel('WATER1.XLS')
train.head()

,region,district_code,population,water_quality,quality_group,quantity,quantity_group,source,region_code
0,Iringa,5,109,soft,good,enough,enough,spring,11
1,Mara,2,280,soft,good,insufficient,insufficient,rainwater harvesting,20
2,Manyara,4,250,soft,good,enough,enough,dam,21
3,Mtwara,63,58,soft,good,dry,dry,machine dbh,90
4,Kagera,1,0,soft,good,seasonal,seasonal,rainwater harvesting,18


In [3]:
train.drop(['district_code' , 'quality_group' , 'quantity_group','source'  ], axis=1 , inplace=True)
train.head()

,region,population,water_quality,quantity,region_code
0,Iringa,109,soft,enough,11
1,Mara,280,soft,insufficient,20
2,Manyara,250,soft,enough,21
3,Mtwara,58,soft,dry,90
4,Kagera,0,soft,seasonal,18


In [4]:

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

encoder = LabelEncoder()
train.iloc[:,0] = encoder.fit_transform(train.iloc[:,0])
train.iloc[:,3] = encoder.fit_transform(train.iloc[:,3])
train.iloc[:,2] = encoder.fit_transform(train.iloc[:,2])

train.head()

,region,population,water_quality,quantity,region_code
0,3,109,6,1,11
1,9,280,6,2,20
2,8,250,6,1,21
3,12,58,6,0,90
4,4,0,6,3,18


In [5]:
train.head()

,region,population,water_quality,quantity,region_code
0,3,109,6,1,11
1,9,280,6,2,20
2,8,250,6,1,21
3,12,58,6,0,90
4,4,0,6,3,18


In [6]:
from sklearn.model_selection import train_test_split
x_train , x_test , y_train , y_test = train_test_split(train.drop('quantity' , axis=1), train['quantity'] , test_size=0.3 , random_state=101)

In [7]:
from sklearn.linear_model import LogisticRegression
logmodel = LogisticRegression()
logmodel.fit(x_train,y_train)
y_pred=logmodel.predict(x_test)
y_pred

array([1, 1, 1, ..., 1, 1, 1], dtype=int64)

In [8]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))


             precision    recall  f1-score   support

          0       0.00      0.00      0.00      1923
          1       0.56      0.99      0.72     10019
          2       0.40      0.02      0.04      4484
          3       0.00      0.00      0.00      1164
          4       0.00      0.00      0.00       230

avg / total       0.42      0.56      0.41     17820



C:\Users\aniru\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [9]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, max_depth=1000,random_state=3,min_samples_leaf=15)
clf.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=1000, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=15, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=3, verbose=0, warm_start=False)

In [10]:
y_pred=clf.predict(x_test)

In [11]:
print(classification_report(y_test,y_pred))

             precision    recall  f1-score   support

          0       0.64      0.28      0.39      1923
          1       0.63      0.92      0.75     10019
          2       0.60      0.30      0.39      4484
          3       0.57      0.04      0.07      1164
          4       0.61      0.43      0.50       230

avg / total       0.62      0.63      0.57     17820



In [12]:
from sklearn.neural_network import MLPClassifier
mlp=MLPClassifier(hidden_layer_sizes=(50,50,50),activation='relu',solver='adam',max_iter=500)
mlp.fit(x_train,y_train)
predict_train=mlp.predict(x_train)
predict_test=mlp.predict(x_test)

In [13]:
print(classification_report(y_test,predict_test))

             precision    recall  f1-score   support

          0       0.71      0.17      0.27      1923
          1       0.59      0.93      0.73     10019
          2       0.52      0.19      0.27      4484
          3       0.49      0.02      0.04      1164
          4       0.67      0.23      0.34       230

avg / total       0.58      0.59      0.51     17820



In [14]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
seed=7
test_size=0.33
x_train , x_test , y_train , y_test = train_test_split(train.drop('quantity' , axis=1), train['quantity'] , test_size=test_size , random_state=seed)
model=XGBClassifier()
model.fit(x_train,y_train)
print(model)
y_pred=model.predict(x_test)
predictions=[round(value) for value in y_pred]
accuracy=accuracy_score(y_test,predictions)
print("accuracy: %.2f%%" %(accuracy * 100.0))

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=3, min_child_weight=1, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='multi:softprob', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1)
accuracy: 60.36%


In [15]:
from lightgbm import LGBMClassifier

lgbm = LGBMClassifier(objective='multiclass', random_state=5)

lgbm.fit(x_train, y_train)

y_pred = lgbm.predict(x_test)

In [16]:
print(classification_report(y_test,y_pred))

             precision    recall  f1-score   support

          0       0.65      0.27      0.38      2043
          1       0.62      0.92      0.74     10907
          2       0.60      0.28      0.38      5002
          3       0.61      0.04      0.08      1384
          4       0.61      0.46      0.52       266

avg / total       0.62      0.62      0.56     19602



In [ ]:
from sklearn.svm import SVC
clf = SVC(kernel='linear' )
clf.fit(x_train, y_train)  

y_pred=clf.predict(x_test)

In [ ]:
print(classification_report(y_test,y_pred))